First set up code to visualize a sound form

In [169]:
%pylab inline
import IPython.display as ipd
import librosa
from librosa import load, display
import glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()
import pickle
from common import save_as_pickle
# import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['display', 'load']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [170]:
import pandas as pd
import numpy as np

In [171]:
# You should change these paths according to the path of the files on your system.
PATH_TO_TRAIN_LABELS = "data/train/train.csv"
PATH_TO_TEST_LABELS = "data/test/test.csv"
PATH_TO_TRAIN_AUDIO_FILES = "data/train/wav/"
PATH_TO_TEST_AUDIO_FILES = "data/test/wav/"
PATH_TO_SUBMISSION = "submission/"
PATH_TO_PICKLE = "pickles/"
SUBMISSION_TITLE = "nn chroma"

In [172]:
# It is easier to deal with csv if you can load it into a structure you can work with.
# Pandas are the most convenient way to do that and are available with 
# inbuilt functionality to handle csv file.

# Pandas assumes that the first row in your file is the header adn not the actual values.
# This behavior can be overriden by passing header=None as a parameter.
train = pd.read_csv(PATH_TO_TRAIN_LABELS)
test = pd.read_csv(PATH_TO_TEST_LABELS)

In [173]:
# You can reactivate this cell to make sure your model is working correctly in terms of dimensions.
#train = train[:2]
#test = test[:2]

In [174]:
train_error_count = 0
train_error_labels = []
test_error_count = 0
test_error_labels = []

In [175]:
# To start with classification, we first need to convert the wav sound files into a format we can work 
# with. It is easier to take the amplitude at each sampling point and use that 
# numeric value to form a feature vector.
def train_parser(row):
    global train_error_count
    global train_error_labels
    path_to_wav_files = PATH_TO_TRAIN_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path)
        stft = np.abs(librosa.stft(data))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T,axis=0)
    except Exception as ex:
        print(ex)
        train_error_count += 1
        train_error_labels.append(row.ID)
        return [0]*12, row.Class
    features = chroma
    label = row.Class
    return [features, label]

In [176]:
# To create the training feature matrix, we can apply our parser to each training sample.
train_features = train.progress_apply(train_parser,axis=1)
print("%d samples had errors while parsing" % train_error_count)
print("Errorneous samples", train_error_labels)
save_as_pickle(data=train_features,pickle_file=PATH_TO_PICKLE + SUBMISSION_TITLE + " train.pickle" )

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')



0 samples had errors while parsing
Errorneous samples []


In [177]:
# Renaming the columns to singnify what they mean helps with documentation,
# and also helps you keep track of them later on.
train_features.columns = ['feature','label']
# train_features.head()

In [178]:
# this library helps us convert string labels into easy to handle encoded labels.
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [179]:
X = np.array(train_features.feature.tolist())
Y = np.array(train_features.label.tolist())
lb = LabelEncoder()
# Since labels are categories they dont inherently have an order amongst themselves.
# For example, Apples > oranges does not make any sense. So to madel such categorical 
# variables, we can convert them to one hot vectors.
Y = to_categorical(lb.fit_transform(Y))

In [180]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [181]:
number_of_labels = Y.shape[1]
filter_size = 2

In [182]:
model = Sequential()

In [183]:
model.add(Dense(256, input_shape=(12,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [184]:
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [185]:
model.add(Dense(number_of_labels))
model.add(Activation('softmax'))

In [186]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 256)               3328      
_________________________________________________________________
activation_13 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_14 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                2570      
__________

In [187]:
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')


In [188]:
model.fit(X,Y, batch_size=32, epochs=50)

Epoch 1/50
5435/5435 [==============================] - ETA: 1:41 - loss: 2.2974 - acc: 0.093 - ETA: 13s - loss: 2.3241 - acc: 0.097 - ETA: 6s - loss: 2.3112 - acc: 0.1074 - ETA: 4s - loss: 2.3087 - acc: 0.106 - ETA: 3s - loss: 2.3064 - acc: 0.095 - ETA: 2s - loss: 2.2962 - acc: 0.100 - ETA: 2s - loss: 2.2852 - acc: 0.099 - ETA: 2s - loss: 2.2862 - acc: 0.097 - ETA: 1s - loss: 2.2818 - acc: 0.095 - ETA: 1s - loss: 2.2768 - acc: 0.099 - ETA: 1s - loss: 2.2753 - acc: 0.105 - ETA: 1s - loss: 2.2701 - acc: 0.106 - ETA: 0s - loss: 2.2724 - acc: 0.107 - ETA: 0s - loss: 2.2673 - acc: 0.109 - ETA: 0s - loss: 2.2648 - acc: 0.112 - ETA: 0s - loss: 2.2627 - acc: 0.115 - ETA: 0s - loss: 2.2606 - acc: 0.119 - ETA: 0s - loss: 2.2591 - acc: 0.121 - ETA: 0s - loss: 2.2587 - acc: 0.123 - ETA: 0s - loss: 2.2575 - acc: 0.125 - ETA: 0s - loss: 2.2527 - acc: 0.130 - ETA: 0s - loss: 2.2512 - acc: 0.133 - 2s 317us/step - loss: 2.2518 - acc: 0.1332
Epoch 2/50
5435/5435 [==============================] - ETA: 

5435/5435 [==============================] - ETA: 2s - loss: 1.6275 - acc: 0.406 - ETA: 1s - loss: 1.7258 - acc: 0.350 - ETA: 0s - loss: 1.7384 - acc: 0.358 - ETA: 0s - loss: 1.7082 - acc: 0.346 - ETA: 0s - loss: 1.7135 - acc: 0.342 - ETA: 0s - loss: 1.6958 - acc: 0.348 - ETA: 0s - loss: 1.7162 - acc: 0.341 - ETA: 0s - loss: 1.7134 - acc: 0.347 - ETA: 0s - loss: 1.7101 - acc: 0.343 - ETA: 0s - loss: 1.7098 - acc: 0.349 - ETA: 0s - loss: 1.7097 - acc: 0.348 - ETA: 0s - loss: 1.7088 - acc: 0.347 - ETA: 0s - loss: 1.7097 - acc: 0.349 - ETA: 0s - loss: 1.7135 - acc: 0.351 - ETA: 0s - loss: 1.7137 - acc: 0.350 - ETA: 0s - loss: 1.7092 - acc: 0.355 - ETA: 0s - loss: 1.7057 - acc: 0.356 - ETA: 0s - loss: 1.7019 - acc: 0.360 - ETA: 0s - loss: 1.7032 - acc: 0.363 - ETA: 0s - loss: 1.7031 - acc: 0.363 - ETA: 0s - loss: 1.7019 - acc: 0.363 - 1s 202us/step - loss: 1.6971 - acc: 0.3663
Epoch 11/50
5435/5435 [==============================] - ETA: 1s - loss: 1.8280 - acc: 0.312 - ETA: 1s - loss: 1.6

5435/5435 [==============================] - ETA: 1s - loss: 1.4980 - acc: 0.375 - ETA: 1s - loss: 1.6220 - acc: 0.371 - ETA: 1s - loss: 1.5801 - acc: 0.408 - ETA: 1s - loss: 1.5824 - acc: 0.411 - ETA: 0s - loss: 1.6124 - acc: 0.405 - ETA: 0s - loss: 1.5851 - acc: 0.418 - ETA: 0s - loss: 1.5738 - acc: 0.419 - ETA: 0s - loss: 1.5763 - acc: 0.420 - ETA: 0s - loss: 1.5681 - acc: 0.426 - ETA: 0s - loss: 1.5690 - acc: 0.429 - ETA: 0s - loss: 1.5687 - acc: 0.428 - ETA: 0s - loss: 1.5674 - acc: 0.425 - ETA: 0s - loss: 1.5549 - acc: 0.429 - ETA: 0s - loss: 1.5486 - acc: 0.430 - ETA: 0s - loss: 1.5524 - acc: 0.426 - ETA: 0s - loss: 1.5495 - acc: 0.428 - ETA: 0s - loss: 1.5517 - acc: 0.427 - ETA: 0s - loss: 1.5469 - acc: 0.428 - ETA: 0s - loss: 1.5452 - acc: 0.428 - ETA: 0s - loss: 1.5536 - acc: 0.425 - ETA: 0s - loss: 1.5506 - acc: 0.427 - ETA: 0s - loss: 1.5491 - acc: 0.427 - ETA: 0s - loss: 1.5547 - acc: 0.425 - ETA: 0s - loss: 1.5557 - acc: 0.425 - 1s 239us/step - loss: 1.5552 - acc: 0.4267


5435/5435 [==============================] - ETA: 1s - loss: 1.4154 - acc: 0.312 - ETA: 0s - loss: 1.4772 - acc: 0.457 - ETA: 0s - loss: 1.4865 - acc: 0.444 - ETA: 0s - loss: 1.5185 - acc: 0.441 - ETA: 0s - loss: 1.5057 - acc: 0.445 - ETA: 0s - loss: 1.5180 - acc: 0.447 - ETA: 0s - loss: 1.5097 - acc: 0.453 - ETA: 0s - loss: 1.5126 - acc: 0.445 - ETA: 0s - loss: 1.5017 - acc: 0.451 - ETA: 0s - loss: 1.4965 - acc: 0.451 - ETA: 0s - loss: 1.4880 - acc: 0.452 - ETA: 0s - loss: 1.4930 - acc: 0.453 - ETA: 0s - loss: 1.4887 - acc: 0.455 - ETA: 0s - loss: 1.4985 - acc: 0.452 - ETA: 0s - loss: 1.4957 - acc: 0.451 - ETA: 0s - loss: 1.4933 - acc: 0.454 - ETA: 0s - loss: 1.4933 - acc: 0.454 - ETA: 0s - loss: 1.4908 - acc: 0.458 - ETA: 0s - loss: 1.4887 - acc: 0.460 - ETA: 0s - loss: 1.4896 - acc: 0.460 - ETA: 0s - loss: 1.4888 - acc: 0.459 - ETA: 0s - loss: 1.4915 - acc: 0.459 - 1s 220us/step - loss: 1.4918 - acc: 0.4600
Epoch 27/50
5435/5435 [==============================] - ETA: 0s - loss: 1.3

5435/5435 [==============================] - ETA: 1s - loss: 1.2869 - acc: 0.593 - ETA: 1s - loss: 1.3729 - acc: 0.513 - ETA: 1s - loss: 1.3851 - acc: 0.501 - ETA: 1s - loss: 1.3543 - acc: 0.522 - ETA: 0s - loss: 1.3674 - acc: 0.517 - ETA: 0s - loss: 1.3789 - acc: 0.512 - ETA: 0s - loss: 1.3738 - acc: 0.510 - ETA: 0s - loss: 1.3871 - acc: 0.502 - ETA: 0s - loss: 1.3879 - acc: 0.500 - ETA: 0s - loss: 1.3819 - acc: 0.501 - ETA: 0s - loss: 1.3903 - acc: 0.498 - ETA: 0s - loss: 1.4010 - acc: 0.491 - ETA: 0s - loss: 1.3996 - acc: 0.491 - ETA: 0s - loss: 1.4054 - acc: 0.488 - ETA: 0s - loss: 1.4086 - acc: 0.488 - ETA: 0s - loss: 1.4126 - acc: 0.487 - ETA: 0s - loss: 1.4190 - acc: 0.486 - ETA: 0s - loss: 1.4232 - acc: 0.483 - ETA: 0s - loss: 1.4223 - acc: 0.483 - ETA: 0s - loss: 1.4299 - acc: 0.482 - ETA: 0s - loss: 1.4334 - acc: 0.481 - ETA: 0s - loss: 1.4369 - acc: 0.479 - ETA: 0s - loss: 1.4383 - acc: 0.478 - 1s 226us/step - loss: 1.4375 - acc: 0.4793
Epoch 35/50
5435/5435 [===============

5435/5435 [==============================] - ETA: 1s - loss: 1.5505 - acc: 0.406 - ETA: 1s - loss: 1.4390 - acc: 0.486 - ETA: 1s - loss: 1.3974 - acc: 0.494 - ETA: 1s - loss: 1.3698 - acc: 0.491 - ETA: 1s - loss: 1.3681 - acc: 0.486 - ETA: 1s - loss: 1.3814 - acc: 0.485 - ETA: 1s - loss: 1.3862 - acc: 0.488 - ETA: 1s - loss: 1.3791 - acc: 0.494 - ETA: 0s - loss: 1.3670 - acc: 0.497 - ETA: 0s - loss: 1.3618 - acc: 0.500 - ETA: 0s - loss: 1.3759 - acc: 0.489 - ETA: 0s - loss: 1.3832 - acc: 0.489 - ETA: 0s - loss: 1.3891 - acc: 0.485 - ETA: 0s - loss: 1.3921 - acc: 0.487 - ETA: 0s - loss: 1.4012 - acc: 0.484 - ETA: 0s - loss: 1.4046 - acc: 0.485 - ETA: 0s - loss: 1.4105 - acc: 0.481 - ETA: 0s - loss: 1.4050 - acc: 0.484 - ETA: 0s - loss: 1.3997 - acc: 0.486 - ETA: 0s - loss: 1.3980 - acc: 0.487 - ETA: 0s - loss: 1.3987 - acc: 0.488 - ETA: 0s - loss: 1.3988 - acc: 0.488 - ETA: 0s - loss: 1.4024 - acc: 0.486 - ETA: 0s - loss: 1.4010 - acc: 0.487 - ETA: 0s - loss: 1.4024 - acc: 0.486 - ETA: 

5435/5435 [==============================] - ETA: 0s - loss: 1.3319 - acc: 0.531 - ETA: 1s - loss: 1.3933 - acc: 0.489 - ETA: 1s - loss: 1.3895 - acc: 0.482 - ETA: 1s - loss: 1.3623 - acc: 0.497 - ETA: 1s - loss: 1.3836 - acc: 0.486 - ETA: 1s - loss: 1.3968 - acc: 0.482 - ETA: 1s - loss: 1.3802 - acc: 0.493 - ETA: 1s - loss: 1.3883 - acc: 0.500 - ETA: 0s - loss: 1.3915 - acc: 0.498 - ETA: 0s - loss: 1.3865 - acc: 0.499 - ETA: 0s - loss: 1.3836 - acc: 0.497 - ETA: 0s - loss: 1.3801 - acc: 0.497 - ETA: 0s - loss: 1.3709 - acc: 0.499 - ETA: 0s - loss: 1.3658 - acc: 0.499 - ETA: 0s - loss: 1.3646 - acc: 0.500 - ETA: 0s - loss: 1.3646 - acc: 0.503 - ETA: 0s - loss: 1.3680 - acc: 0.499 - ETA: 0s - loss: 1.3645 - acc: 0.500 - ETA: 0s - loss: 1.3632 - acc: 0.500 - ETA: 0s - loss: 1.3615 - acc: 0.501 - ETA: 0s - loss: 1.3628 - acc: 0.503 - ETA: 0s - loss: 1.3670 - acc: 0.501 - ETA: 0s - loss: 1.3698 - acc: 0.500 - ETA: 0s - loss: 1.3686 - acc: 0.502 - ETA: 0s - loss: 1.3692 - acc: 0.501 - 1s 24

In [189]:
def test_parser(row):
    global test_error_count
    global test_error_labels
    path_to_wav_files = PATH_TO_TEST_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path)
        stft = np.abs(librosa.stft(data))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T,axis=0)
    except Exception as ex:
        test_error_count += 1
        test_error_labels.append(row.ID)
        return pd.Series([0]*12)
    features = chroma
    return pd.Series(features)

In [190]:
test_features = test.progress_apply(test_parser,axis=1, reduce = True)
print("%d samples had errors while parsing" % test_error_count)
print("Errorneous samples", test_error_labels)
save_as_pickle(data=train_features,pickle_file=PATH_TO_PICKLE + SUBMISSION_TITLE + " test.pickle" )

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')



0 samples had errors while parsing
Errorneous samples []


In [191]:
X_test = test_features
# X_test

In [192]:
test_labels = model.predict(X_test, batch_size=32)

In [193]:
test_labels_strings = lb.inverse_transform(test_labels.argmax(axis=1))
# test_labels_strings

C:\Users\shrung\AppData\Local\Continuum\miniconda3\envs\deeplearning\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [194]:
test['Class'] = test_labels_strings

In [195]:
test.to_csv(PATH_TO_SUBMISSION + SUBMISSION_TITLE + ".csv",index=None)

This approach gives 56% accuracy with the above setup.